In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import cv2
import re
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions, ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.keras.models import Sequential 
from sklearn.model_selection import train_test_split
from tensorflow.keras import optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
import os
import tensorflow as tf



In [ ]:
sam_sub = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv')

In [ ]:
sam_sub

In [ ]:
train_dir = '/kaggle/input/plant-pathology-2021-fgvc8/train_images'
test_dir = '/kaggle/input/plant-pathology-2021-fgvc8/test_images'

In [ ]:
train = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
# traincom= train.join(pd.get_dummies(train.labels))

In [ ]:
# traindum = traincom.drop('labels', axis = 1)

In [ ]:
# traindum

In [ ]:
#get ids in order in the directory
#get train file paths
test_filepaths = []

test_ids = []

for dirname, _, filenames in os.walk(test_dir):
    for filename in filenames:
        test_ids.append(filename)
        test_filepaths.append(os.path.join(dirname, filename))

In [ ]:
test_df = pd.DataFrame(test_ids,columns = ['image'])

In [ ]:
test_df

In [ ]:
train_datagen_sub = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)

In [ ]:
train_generator_sub = train_datagen_sub.flow_from_dataframe(
        train,
        directory = train_dir,
        x_col = 'image',
        y_col = 'labels',
        target_size = (432, 648),
        batch_size = 16,
    
        class_mode = 'categorical')

In [ ]:
test_datagen = ImageDataGenerator()

In [ ]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col="image",
    target_size=(432, 648),
    batch_size=1,
    class_mode=None,
    shuffle=False,
)

In [ ]:
# trained_model_sub = tf.keras.models.load_model('../input/pp21model/pp21_sub')

In [ ]:
from tensorflow.keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
#trained_model_sub = tf.keras.models.load_model('../input/effnet/pp21_effnet_subh')
#trained_model_sub = tf.keras.models.load_model('../input/effnet2/pp21_top_fit_sub_effnet2')
#trained_model_sub = tf.keras.models.load_model('../input/effnet5/pp21_effnet_sub5')
trained_model_sub = tf.keras.models.load_model('../input/effnettop/pp21_top_fit_effnet10',custom_objects = {'f1_m':f1_m})

In [ ]:
y_pred = trained_model_sub.predict(test_generator)

In [ ]:
# y_pred_idxs = [np.argmax(i) for i in y_pred]
# y_pred_idxs

In [ ]:
predicted_class_indices = np.argmax(y_pred, axis = 1)

In [ ]:
labels = train_generator_sub.class_indices
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
sub = pd.DataFrame({'image': test_ids, 'labels': predictions})

In [ ]:
# y_pred_max = y_pred.copy()

In [ ]:
# for i in range(len(y_pred_idxs)):
#     y_pred_max[i][y_pred_idxs[i]] = 1
#     for j in range(12):
#         if y_pred_max[i][j] != 1:
#             y_pred_max[i][j] = 0

In [ ]:
# y_pred_df_max = pd.DataFrame(y_pred_max, index = test_ids, columns=traindum.set_index('image').columns, dtype='int64')

In [ ]:
# y_pred_df_max

In [ ]:
# sub_df = pd.DataFrame(y_pred_df_max.idxmax(1)).reset_index()

In [ ]:
# sub_df.columns = ['image', 'labels']

In [ ]:
# sub_df

In [ ]:
# sub_df.to_csv('submission.csv', index=False)

In [ ]:
sub.to_csv('submission.csv', index=False)